In [1]:
import pickle

with open('yolov5l6+clip/1_dictionary_full_train.p', 'rb') as f:
    dictionary = pickle.load(f)
    
dictionary

{0: {'image_emb': tensor([[ 0.0221,  0.5029, -0.3162,  ...,  1.0371,  0.4727, -0.2151],
          [-0.1107,  0.2563,  0.1511,  ...,  1.2871, -0.0994, -0.0670],
          [ 0.1321,  0.4282, -0.2360,  ...,  1.2500, -0.0720, -0.0065],
          [-0.1364,  0.4077, -0.5093,  ...,  0.9365, -0.0548, -0.2174],
          [-0.2417,  0.2808, -0.0729,  ...,  1.1367,  0.1469, -0.1929],
          [-0.1302,  0.2028, -0.2114,  ...,  1.1064,  0.1814, -0.1130]],
         dtype=torch.float16),
  'text_emb': tensor([[-0.1068, -0.1234, -0.4180,  ...,  0.0963, -0.0219, -0.4702],
          [ 0.0802, -0.3291, -0.4617,  ..., -0.0427,  0.0495, -0.2220]],
         dtype=torch.float16),
  'text_similarity': tensor([[1.3977e-01, 4.0894e-03, 7.6408e-03, 7.3242e-01, 3.3069e-04, 1.1584e-01],
          [9.4434e-01, 3.4393e-02, 1.0658e-02, 2.7239e-05, 5.2512e-05, 1.0658e-02]],
         dtype=torch.float16),
  'df_boxes':          xmin        ymin        xmax        ymax  confidence  class  \
  0    0.465897   41.905228

# BASELINE with yolov5l6

In this assessment, we are analyzing the baseline performance. The YOLOv5l6 model was utilized to locate the boxes, while the corresponding text for these boxes was computed in advance using CLIP.

In [74]:
import torch
import torchvision

import pickle
from tqdm import tqdm

In [122]:
def load_the_best_matching_box(file_name):
    box_target = []
    box_pred = []
    
    with open(file_name, 'rb') as f:
        dictionary = pickle.load(f)
        
    for sample in tqdm(dictionary.keys()):
        
        try:
            # get the sample
            element = dictionary[sample]
            
            for sim, cap in zip(element['text_similarity'],element['caption']):
                
                # get the max similarity index
                max_sim_index = torch.argmax(sim).numpy()
                
                # gete the  bbox_target
                box_pred.append(element['df_boxes'].loc[max_sim_index][:4].values.astype('int'))
                box_target.append(element['bbox_target'])
        except:
            KeyError: print('error in sample:', sample)
            
    box_pred = torch.stack([torch.Tensor(p) for p in box_pred])
    box_target = torch.tensor(box_target)
    
    # convert the box_pred to x1, y1, x2, y2
    box_target[:, 2] = box_target[:, 0] + box_target[:, 2]
    box_target[:, 3] = box_target[:, 1] + box_target[:, 3]
        
    return box_pred, box_target

# 'yolov5l6+clip/1_dictionary_full_train.p'
box_pred, box_target = load_the_best_matching_box('yolov5l6+clip/1_dictionary_full_test.p')

100%|██████████| 42226/42226 [00:07<00:00, 5824.72it/s]


In [119]:
box_pred,box_target

(tensor([[374.,  68., 510., 262.],
         [374.,  68., 510., 262.],
         [426., 107., 639., 237.],
         ...,
         [199., 271., 333., 471.],
         [225.,   0., 330., 127.],
         [ 61.,   0., 239.,  82.]]),
 tensor([[374.3100,  65.0600, 510.3500, 267.0000],
         [374.3100,  65.0600, 510.3500, 267.0000],
         [ 93.9500,  83.2900, 598.5600, 373.8600],
         ...,
         [348.5800, 230.9100, 480.0000, 471.1400],
         [212.8200,   0.0000, 355.9000, 132.4100],
         [212.8200,   0.0000, 355.9000, 132.4100]]))

In [77]:
print('assert same dimensionality',len(box_pred) == len(box_target))

# convert the lists to tensors
# torchvision.ops.box_iou(box_target, box_pred)

assert same dimensionality True


In [7]:
box_target.shape,  box_pred.shape

(torch.Size([74895, 4]), torch.Size([74895, 4]))

In [124]:
mean_iou = []

def iou(box1, box2):
    # get the coordinates of bounding boxes
    b1_x1, b1_y1, b1_x2, b1_y2 = box1
    b2_x1, b2_y1, b2_x2, b2_y2 = box2
    
    # get the corrdinates of the intersection rectangle
    inter_rect_x1 =  max(b1_x1, b2_x1)
    inter_rect_y1 =  max(b1_y1, b2_y1)
    inter_rect_x2 =  min(b1_x2, b2_x2)
    inter_rect_y2 =  min(b1_y2, b2_y2)
    
    # Intersection area
    inter_area = max(inter_rect_x2 - inter_rect_x1 + 1, 0) * max(inter_rect_y2 - inter_rect_y1 + 1, 0)
    
    # Union Area
    b1_area = (b1_x2 - b1_x1 + 1)*(b1_y2 - b1_y1 + 1)
    b2_area = (b2_x2 - b2_x1 + 1)*(b2_y2 - b2_y1 + 1)
    
    iou = inter_area / (b1_area + b2_area - inter_area)
    
    return iou
    
for b_t, b_p in zip(box_target, box_pred):
    
    # get the iou
    mean_iou.append(iou(b_t, b_p))
    
mean_iou_m = torch.stack(mean_iou).mean().numpy()

print(mean_iou_m)

0.6099380016326904


In [140]:
file_name = 'yolov5l6+clip/1_dictionary_full_test.p'

with open(file_name, 'rb') as f:
    dictionary = pickle.load(f)

print('maximum number of boxes found: ',max([len(dictionary[sample]['image_emb']) for sample in dictionary.keys()]))
print('mean number of boxes found: ',np.array([len(dictionary[sample]['image_emb']) for sample in dictionary.keys()]).mean())
print('std number of boxes found: ',np.array([len(dictionary[sample]['image_emb']) for sample in dictionary.keys()]).std())
print('count max number of boxes found: ',[len(dictionary[sample]['image_emb']) for sample in dictionary.keys()].count(32))
print('count of number of boxes found:')
print([[len(dictionary[sample]['image_emb']) for sample in dictionary.keys()].count(xx+1) for xx in range(32)])

maximum number of boxes found:  27
mean number of boxes found:  5.586103921958989
std number of boxes found:  2.7619218371402146
count max number of boxes found:  0
count of number of boxes found:
[25, 201, 794, 1047, 954, 637, 449, 290, 206, 132, 86, 49, 52, 31, 30, 9, 17, 3, 2, 0, 1, 0, 4, 2, 0, 1, 1, 0, 0, 0, 0, 0]


In [141]:
file_name = 'yolov5l6+clip/1_dictionary_full_train.p'

with open(file_name, 'rb') as f:
    dictionary = pickle.load(f)

print('maximum number of boxes found: ',max([len(dictionary[sample]['image_emb']) for sample in dictionary.keys()]))
print('mean number of boxes found: ',np.array([len(dictionary[sample]['image_emb']) for sample in dictionary.keys()]).mean())
print('std number of boxes found: ',np.array([len(dictionary[sample]['image_emb']) for sample in dictionary.keys()]).std())
print('count max number of boxes found: ',[len(dictionary[sample]['image_emb']) for sample in dictionary.keys()].count(32))
print('count of number of boxes found:')
print([[len(dictionary[sample]['image_emb']) for sample in dictionary.keys()].count(xx+1) for xx in range(32)])

maximum number of boxes found:  32
mean number of boxes found:  5.56635722066973
std number of boxes found:  2.7148657554979914
count max number of boxes found:  3
count of number of boxes found:
[248, 1577, 6883, 8627, 7880, 5624, 3697, 2583, 1698, 1039, 776, 498, 341, 283, 144, 98, 56, 50, 36, 28, 17, 14, 9, 6, 2, 9, 0, 0, 0, 0, 0, 3]
